# Tunga Amazon

*Pesquisar no Amazon os seguintes produtos* <br>

- flipper zero
- Tv box 
- bloqueador de sinal
- carregador de bateria de celular
- jammer
- reforçador de celular
- bateria de celular
- transmissor fm

A pesquisa também deve registrar a quantidade de produtos localizados em cada plataforma com a respectiva  palvra-chave

In [33]:
produtos_para_pesquisa = [
#    'flipper zero',
#     'Tv box',
#     'bloqueador de sinal',
#     'carregador de bateria de celular',
#     'jammer bloqueador',
#     'reforçador de celular',
#     'bateria de celular',
     'transmissor fm']#
# ]

### importando bibliotecas

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

from time import sleep
from datetime import datetime
import re

### configurações do web browser

In [3]:
#funções do webdriver
def get_driver() -> webdriver:
    """
    Retorna um objeto WebDriver para o Chrome.
    """

    # create object for chrome options

    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox') 
    chrome_options.add_argument('disable-notifications') 
    chrome_options.add_argument('--disable-infobars') 
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument("--disable-translate")
    chrome_options.add_argument("--disable-web-security")
    chrome_options.add_argument("--disable-logging")
    chrome_options.add_argument("--disable-plugins-discovery")

    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    chrome_options.add_argument(f"user-agent={user_agent}")


    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),
                                  options = chrome_options)  
    
    driver.maximize_window()# Maximiza a janela
    driver.implicitly_wait(3)
    
    return driver

def search(driver: webdriver, query: str, element_id: str) -> None:
    """
    Busca por query no element_id informado.
    """
    search_box = driver.find_element(By.ID, element_id)
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)

### capturar as quantidades de cada produto

In [ ]:
driver = get_driver() # abre o navegador

# acessar amazon


quantidade_produtos = []

for i in produtos_para_pesquisa:
    element_id = 'twotabsearchtextbox'
    url = 'https://www.amazon.com.br'
    driver.get(url)
    sleep(2)

    search(driver, i, element_id)
    sleep(2)    

    try:
        quantidade =  driver.find_elements(
            By.CLASS_NAME,'s-pagination-disabled'
                )[-1].get_attribute('innerText')

        quantidade = int(quantidade) * 52
    except: 
        quantidade = 0
    
    
    # Obter a data e hora atuais
    data_hora_atual = datetime.now()
    data_hora_atual = f'{data_hora_atual.day}/{data_hora_atual.month}/{data_hora_atual.year} {data_hora_atual.hour}:{data_hora_atual.minute}:{data_hora_atual.second}'
    
    quantidade_produtos.append({'plataforma' :'Amazon',
                                'produto'    : i,
                                'quantidade' : quantidade,
                                'data_coleta': data_hora_atual
                               })

driver.quit()

### salvar resultado das quantidades

In [ ]:
df = pd.DataFrame(quantidade_produtos)
df.to_csv('quantidades_amazon.csv', index = None, sep = ';')
df

### capturar os links dos produtos

In [34]:
driver = get_driver()
links_produtos = set() # vai guardar os links dos produtos

for produto_pesquisado in produtos_para_pesquisa:
    driver.get('https://www.amazon.com.br')
    sleep(3)

    search(driver = driver, query = produto_pesquisado, element_id = 'twotabsearchtextbox')
    sleep(3)
    
    ## descobre a úlitim página 
    try:
        ultima_pagina = driver.find_elements(By.CLASS_NAME, 's-pagination-disabled')[1].get_attribute('innerHTML')
        ultima_pagina = int(ultima_pagina)
    except: 
        ultima_pagina = 1
            

    for pagina in range(ultima_pagina):
        if True:
        #try:
            print('pesquisando página ', pagina + 1, 'de', ultima_pagina)
            #descobrir os links dos produtos

            a_elements = driver.find_elements(By.CLASS_NAME, 'a-link-normal')
            # a_elements = a_elements.find_elements(By.TAG_NAME, 'a')
            for element in a_elements:

                link_produto = element.get_attribute('href').split('/ref=')[0]

                # termos isentam os links (se tiver esses termos, não captura o link):
                # '/x/c/','bestsellers','goldbox'
                # termos que inserem os links '/gp/',  '/dp/'
                termos_blacklist = ['/x/c/', 'bestsellers', 'goldbox','offer-listing','kindle-dbs','/help/','/slredirect/']
                termos_whitelist = ['/gp/', '/dp/']

                if all(termo not in link_produto for termo in termos_blacklist) and any(termo in link_produto for termo in termos_whitelist):
                    links_produtos.add(link_produto)

            try:
                driver.find_elements(By.CLASS_NAME, 's-pagination-next')[0].click()
                sleep(3)
            except:
                pass

        #except Exception as e:
            #print(e)


driver.quit()

pesquisando página  1 de 7
pesquisando página  2 de 7
pesquisando página  3 de 7
pesquisando página  4 de 7
pesquisando página  5 de 7
pesquisando página  6 de 7
pesquisando página  7 de 7


In [29]:
ultima_pagina

7

## navegar pelos produtos

In [35]:
len(links_produtos)

291

In [36]:
driver = get_driver() # abre o navegador

final = len(links_produtos)
produtos = []

for link in links_produtos:
    final = final -1
    if final % 10 == 0:
        print(final)

    driver.get(link)

    try:
        titulo_produto = driver.find_element(By.ID, 'productTitle').text
    except:
        titulo_produto = ''

    try:
        id_vendedor    = driver.find_element(By.XPATH, '//*[@id="tabular-buybox"]/div[1]/div[6]/div').get_attribute('innerText')

    except:
        try:
             id_vendedor = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[3]/div[1]/div[8]/div[1]/div/div/div/div[1]/div/div/div/div/div/div[3]/div/div[1]/form[1]/div/table/tbody/tr[2]/td[2]/span').get_attribute('innerText')

        except:
            try:
                id_vendedor = driver.find_element(By.CSS_SELECTOR, '#Ebooks-desktop-printSoldBy > td:nth-child(2) > span').get_attribute('innerText')

            except:
                id_vendedor    = ''

    try:      

        inteiro = driver.find_element(By.CSS_SELECTOR,
                                    '#corePriceDisplay_desktop_feature_div > div.a-section.a-spacing-none.aok-align-center > span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay > span:nth-child(2) > span.a-price-whole'
                                    ).get_attribute('innerText').replace('\n', '')
        fracao = driver.find_element(By.CSS_SELECTOR,
                                    '#corePriceDisplay_desktop_feature_div > div.a-section.a-spacing-none.aok-align-center > span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay > span:nth-child(2) > span.a-price-fraction'
                                    ).get_attribute('innerText')

        preco = f'{inteiro}{fracao}'

    except Exception as e:

        try: 

            inteiro = driver.find_element(By.CSS_SELECTOR,
                                    '#corePrice_feature_div > div > span.a-price.aok-align-center > span:nth-child(2) > span.a-price-whole'
                                     ).get_attribute('innerText')
            fracao =  driver.find_element(By.CSS_SELECTOR, 
                                          '#corePrice_feature_div > div > span.a-price.aok-align-center > span:nth-child(2) > span.a-price-fraction'
                                       ).get_attribute('innerText')

            preco = f'{inteiro},{fracao}'

        except:

            try:
                preco = driver.find_element(By.ID, 'kindle-price-column').get_attribute('innerText')[2:]
            except:
                preco = ''


    try:
        descricao_1 = driver.find_element(By.ID, 'productDescription').text
    except:
        descricao_1 = ''
    try:
        descricao_2 = driver.find_element(By.ID, 'bookDescription_feature_div').text
    except:
        descricao_2 = ''
    try:
        descricao_3 = driver.find_element(By.ID, 'prodDetails').text
    except:
        descricao_3 = ''
    try:
        descricao_4 = driver.find_element(By.ID, 'feature-bullets').text
    except:
        descricao_4 = ''

    try:
        descricao_5 = driver.find_element(By.ID, 'detailBulletsWrapper_feature_div').text
    except:
        descricao_5 = ''


    quantidade = 1


    try: 
        marca = driver.find_element(By.PARTIAL_LINK_TEXT, 'Marca').text.split(': ')[1]
    except: 
        marca = ''

    try:
        categoria = driver.find_element(By.CSS_SELECTOR, "#wayfinding-breadcrumbs_feature_div > ul").get_attribute("innerText")
    except:
        categoria = ''

    # Obter a data e hora atuais
    data_hora_atual = datetime.now()
    data_hora_atual = f'{data_hora_atual.day}/{data_hora_atual.month}/{data_hora_atual.year} {data_hora_atual.hour}:{data_hora_atual.minute}:{data_hora_atual.second}'


    produto = {
        'plataforma'        : 'Amazon',
        'produto_pesquisado': 'transmissor fm',
        'titulo_produto'    : titulo_produto,
        'detalhes_produto'  : '',
        'marca'             : marca,
        'modelo'            : '',
        'id_vendedor'       : id_vendedor,
        'localizacao'       : '',
        'preco_minimo'      : preco,
        'preco_maximo'      : '',
        'preco_promo_minimo': '',
        'preco_promo_maximo': '',
        'quantidade'        : quantidade,
        'descricao_1'       : descricao_1,
        'descricao_2'       : descricao_2,
        'descricao_3'       : descricao_3,
        'descricao_4'       : descricao_4,
        'descricao_5'       : descricao_5,
        'homologado'        : '',
        'vendas_anteriores' : '',
        'url'               : link ,
        'data_coleta'       : data_hora_atual

    } 
                
    produtos.append(produto)

driver.quit()


290
280
270
260
250
240
230
220
210
200
190
180
170
160
150
140
130
120
110
100
90
80
70
60
50
40
30
20
10
0


In [37]:
df = pd.DataFrame(produtos)
df.to_csv('resultados_amazon_transmissor_fm.csv', index = None, sep = ';')
df

,plataforma,produto_pesquisado,titulo_produto,detalhes_produto,marca,modelo,id_vendedor,localizacao,preco_minimo,preco_maximo,...,quantidade,descricao_1,descricao_2,descricao_3,descricao_4,descricao_5,homologado,vendas_anteriores,url,data_coleta
0,Amazon,transmissor fm,X8 carregador de carro sem fio compatível com ...,,YARUIKE,,YARUIKE BM Technology LTD,,"80,00",,...,1,Aviso de tensão da bateria do carro multifunci...,,Informações sobre o produto\nDetalhes técnicos...,Aviso de tensão da bateria do carro multifunci...,,,,https://www.amazon.com.br/carregador-compat%C3...,10/6/2023 17:10:51
1,Amazon,transmissor fm,Vifaleno de Módulo Transmissor FM 0.7-9V 88-10...,,Vifaleno,,YufaeleT,,"46,84",,...,1,Recursos:\nFaz o som ao redor do microfone no ...,,Informações sobre o produto\nDetalhes técnicos...,Sobre este item\nFaz o som ao redor do microfo...,,,,https://www.amazon.com.br/Vifaleno-Transmissor...,10/6/2023 17:11:17
2,Amazon,transmissor fm,Mingzhe T2 BT5.0 Transmissor FM sem fio para c...,,Mingzhe,,Maykalin,,"131,79",,...,1,Chamadas mãos-livres BT:\nEmparelhado com tele...,,Informações sobre o produto\nDetalhes técnicos...,Transmissor FM MP3 Player: Module a frequência...,,,,https://www.amazon.com.br/Mingzhe-Transmissor-...,10/6/2023 17:11:42
3,Amazon,transmissor fm,"balikha 1.2""Transmissor de FM LCD para Adaptad...",,balikha,,Balikha,,"47,99",,...,1,Descrição do produto:\n\n1. Suporte a chamada ...,,Informações sobre o produto\nDetalhes técnicos...,Sobre este item\n1. Suporte a chamada mãos-liv...,,,,https://www.amazon.com.br/balikha-Transmissor-...,10/6/2023 17:12:11
4,Amazon,transmissor fm,"LENCENT Transmissor FM Bluetooth 5.0 de Carro,...",,LENCENT,,LencentUS,,"159,00",,...,1,,,Informações sobre o produto\nDetalhes técnicos...,【PD 20W & QC 3.0 Carregamento Rápido】O transmi...,,,,https://www.amazon.com.br/LENCENT-Transmissor-...,10/6/2023 17:13:0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,Amazon,transmissor fm,3 Pcs Transmissor de rádio sem fio para carro ...,,Chunyue,,Kathxyn,,"223,49",,...,1,Características;\n1. O transmissor de rádio do...,,Informações sobre o produto\nDetalhes técnicos...,Sobre este item\n✿【Qualidade de som de alta re...,,,,https://www.amazon.com.br/Transmissor-r%C3%A1d...,10/6/2023 20:19:27
287,Amazon,transmissor fm,Transmissor FM Mp3/wma/wav Bluetooth Carregado...,,,,SmartHomeMS,,"149,90",,...,1,Transmissor FM Mp3/wma/wav Bluetooth Carregado...,,Informações sobre o produto\nDetalhes técnicos...,,,,,https://www.amazon.com.br/Transmissor-Bluetoot...,10/6/2023 20:20:14
288,Amazon,transmissor fm,"Transmissor FM Bluetooth no Carro, QC 3.0 Micr...",,A/R,,,,,,...,1,Descrever:\n1. Receptor bluetooths player mult...,,Informações sobre o produto\nDetalhes técnicos...,Sobre este item\n✙【Carregamento fácil】Seu adap...,,,,https://www.amazon.com.br/Transmissor-Carregad...,10/6/2023 20:21:5
289,Amazon,transmissor fm,"Transmissor FM Bluetooth, Adaptador para Carro...",,LIANLI,,JIAYUKEJI,,"152,70",,...,1,Bluetooth 5.0 e Google Assistant SIRI: O mais ...,,Informações sobre o produto\nDetalhes técnicos...,Bluetooth 5.0 e Google Assistant SIRI: O mais ...,,,,https://www.amazon.com.br/Transmissor-Bluetoot...,10/6/2023 20:21:55


In [ ]:
tabelas = driver.find_elements(By.CLASS_NAME, 'aplus-tech-spec-table')
data = {}

for tabela in tabelas:
    soup = BeautifulSoup(tabela.get_attribute('innerHTML'), 'html.parser')
    # table = soup.find('table')
    rows = soup.find_all('tr')

    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 0:
            key = cells[0].text.strip()
            value = cells[1].text.strip()
            data[key] = value
